In [2]:
import geopandas as gpd
import glob
import os
import rasterio
from rasterio.mask import mask

In [11]:
field_data = gpd.read_file('../dataset2/field_polygons.geojson')
field_data['Name'] = field_data['Name'].str.cat(field_data.index.astype(str), sep = '_')

field_data.head()

,id,Name,geometry
0,1,Pin4_0,"MULTIPOLYGON (((30.17411 49.99641, 30.17705 50..."
1,2,Pin17_1,"MULTIPOLYGON (((30.24050 49.98769, 30.24404 49..."
2,3,Shev7-8-9_2,"MULTIPOLYGON (((30.32204 50.00902, 30.33148 50..."
3,4,Shev10_3,"MULTIPOLYGON (((30.35295 50.01389, 30.35007 50..."
4,5,Tr5_4,"MULTIPOLYGON (((30.33304 50.02297, 30.34076 50..."


In [17]:
spatial_data_path = '../dataset2/spatial_data/'
tif_files = glob.glob(spatial_data_path + '**/*.tif')
num_tif_files = len(tif_files)

print(f'The "{spatial_data_path}" directory contains {num_tif_files} files with the .tif extension.')

The "../dataset2/spatial_data/" directory contains 68 files with the .tif extension.


In [23]:
from pathlib import Path

resampled_path = Path('../dataset2/resampled')

for _, row in field_data.iterrows():
    dir_path = resampled_path / row['Name']
    dir_path.mkdir(parents=True, exist_ok=True)

In [27]:
image_folders = os.listdir(spatial_data_path)

for image_folder_name in os.listdir(spatial_data_path):
    current_image_folder = os.path.join(spatial_data_path, image_folder_name)
    for image_file_name in os.listdir(current_image_folder):
        image_file_path = os.path.join(current_image_folder, image_file_name)

        with rasterio.open(image_file_path) as image:
            for index, row in field_data.iterrows():
                out_path = os.path.join(resampled_path, row['Name'], image_folder_name, image_file_name).replace("\\","/")

                window = image.window(*row.geometry.bounds)

                data = image.read(1, window=window)

                transform = image.window_transform(window)
                out_meta = image.meta.copy()
                out_meta.update({'driver': 'GTiff',
                                 'height': window.height,
                                 'width': window.width,
                                 'transform': transform})

                print(out_path)

                directory = os.path.dirname(out_path)
                os.makedirs(directory, exist_ok=True)

                with rasterio.open(out_path, 'w', **out_meta) as dst:
                    dst.write(data, 1)

../dataset2/resampled/Pin4_0/201504022343_179D/29.566633136000007_49.434589403000004_31.185382877000006_50.486672568.tif
../dataset2/resampled/Pin17_1/201504022343_179D/29.566633136000007_49.434589403000004_31.185382877000006_50.486672568.tif
../dataset2/resampled/Shev7-8-9_2/201504022343_179D/29.566633136000007_49.434589403000004_31.185382877000006_50.486672568.tif
../dataset2/resampled/Shev10_3/201504022343_179D/29.566633136000007_49.434589403000004_31.185382877000006_50.486672568.tif
../dataset2/resampled/Tr5_4/201504022343_179D/29.566633136000007_49.434589403000004_31.185382877000006_50.486672568.tif
../dataset2/resampled/Shev6_5/201504022343_179D/29.566633136000007_49.434589403000004_31.185382877000006_50.486672568.tif
../dataset2/resampled/Tr4_6/201504022343_179D/29.566633136000007_49.434589403000004_31.185382877000006_50.486672568.tif
../dataset2/resampled/Tr3_7/201504022343_179D/29.566633136000007_49.434589403000004_31.185382877000006_50.486672568.tif
../dataset2/resampled/Tr2_